In [0]:
#Block 1 server setup 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
#Block 2 sync colabserver with drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q tensorflow

In [0]:
#ls "drive/ML Project/"

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
#Block 3 import libraries
from keras.layers import Bidirectional, Dense, Embedding, Input, Lambda, LSTM, RepeatVector, TimeDistributed, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model
from scipy import spatial
import tensorflow as tf
import pandas as pd
import numpy as np
import codecs
import csv
import os
import random
SEED = 30
random.seed(SEED)

Using TensorFlow backend.


In [0]:
#block 4 data preprocessing
BASE_DIR = 'drive/ML Project/'
TRAIN_DATA_FILE = BASE_DIR + 'train.csv'#'train_micro.csv'
GLOVE_EMBEDDING = BASE_DIR + 'glove.6B.300d.txt'
VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = 15
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300

texts = []
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts.append(values[3])
        texts.append(values[4])
print('Found %s texts in train.csv' % len(texts))

tokenizer = Tokenizer(MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index #the dict values start from 1 so this is fine with zeropadding
index2word = {v: k for k, v in word_index.items()}
print('Found %s unique tokens' % len(word_index))
sequences = tokenizer.texts_to_sequences(texts)
data_1 = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post',truncating='post')
print('Shape of data tensor:', data_1.shape)
NB_WORDS = (min(tokenizer.num_words, len(word_index)) + 1 ) #+1 for zero padding
#data_1_val = data_1[801000:807000] #select 6000 sentences as validation data

data_x = np.copy(data_1)
for i in range(len(data_x)):
  rd = random.randint(0,(MAX_SEQUENCE_LENGTH-1)//2)
  data_x[i][rd] = word_index['pad']

data_val_y = data_1[775000:783000]
data_train_y = data_1[:775000]

data_val_x = data_x[775000:783000]
data_train_x = data_x[:775000]

def give():
  data_x = np.copy(data_1)
  for i in range(len(data_x)):
    rd = random.randint(1,(MAX_SEQUENCE_LENGTH-1)//2)
    #data_x[i][rd] = word_index['pad']
  data_val_y = data_1[775000:783000]
  data_train_y = data_1[:775000]
  data_val_x = data_x[775000:783000]
  data_train_x = data_x[:775000]
  return data_train_x,data_train_y, data_val_x, data_val_y
data_train_x,data_train_y, data_val_x, data_val_y = give()

Found 808580 texts in train.csv
Found 95596 unique tokens
Shape of data tensor: (808580, 15)


In [0]:
print(word_index['pad'])
print(index2word[10291])
print(random.randint(0,MAX_SEQUENCE_LENGTH-1))
print(data_x[2])

10291
pad
4
[    2     3     1 10291    10 14300 13598     5  4565     0     0     0
     0     0     0]


In [0]:
#block 5 Loading GloVe matrix
'''
mx = 0
idx = 0
for i in range(len(sequences)):
    if mx <len(sequences[i]):
        mx = len(sequences[i]);
        idx = i
print(idx)
print(texts[36111])
'''
def sent_generator(TRAIN_DATA_FILE, chunksize):
    reader = pd.read_csv(TRAIN_DATA_FILE, chunksize=chunksize, iterator=True)
    for df in reader:
        # print(df.shape)
        # df=pd.read_csv(TRAIN_DATA_FILE, iterator=False)
        val3 = df.iloc[:, 3:4].values.tolist()
        val4 = df.iloc[:, 4:5].values.tolist()
        flat3 = [item for sublist in val3 for item in sublist]
        flat4 = [str(item) for sublist in val4 for item in sublist]
        texts = []
        texts.extend(flat3[:])
        texts.extend(flat4[:])

        sequences = tokenizer.texts_to_sequences(texts)
        data_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post',truncating='post')
        yield [data_train, data_train]

embeddings_index = {}
f = open(GLOVE_EMBEDDING, encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

glove_embedding_matrix = np.zeros((NB_WORDS, EMBEDDING_DIM))
for word, i in word_index.items():
    if i < NB_WORDS:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be the word embedding of 'unk'.
            glove_embedding_matrix[i] = embedding_vector
        else:
            glove_embedding_matrix[i] = embeddings_index.get('unk')
print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))



Found 400000 word vectors.
Null word embeddings: 1


In [0]:
#block 6 contruction of model architecture
batch_size = 500
max_len = MAX_SEQUENCE_LENGTH
emb_dim = EMBEDDING_DIM
latent_dim = 512
intermediate_dim = 1024
epsilon_std = 1.0
num_sampled= 500
kl_weight = 0.001

act = ELU()

#y = Input(batch_shape=(None, max_len, NB_WORDS))
x = Input(batch_shape=(None, max_len))
x_embed = Embedding(NB_WORDS, emb_dim, weights=[glove_embedding_matrix],
                            input_length=max_len, trainable=False)(x)
h = Bidirectional(LSTM(intermediate_dim, return_sequences=False, recurrent_dropout=0.2), merge_mode='concat')(x_embed)
#https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/

#h = Bidirectional(LSTM(intermediate_dim, return_sequences=False), merge_mode='concat')(h)
#h = Dropout(0.2)(h)
#h = Dense(intermediate_dim, activation='linear')(h)
#h = act(h)
#h = Dropout(0.2)(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std, seed=SEED)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later
repeated_context = RepeatVector(max_len)
decoder_h = LSTM(intermediate_dim, return_sequences=True, recurrent_dropout=0.2)
decoder_mean = TimeDistributed(Dense(NB_WORDS, activation='linear'))#https://keras.io/layers/wrappers/  #softmax is applied in the seq2seqloss by tf

#https://github.com/keras-team/keras/issues/5203
h_decoded = decoder_h(repeated_context(z))
x_decoded_mean = decoder_mean(h_decoded)

#https://stackoverflow.com/questions/46663013/what-is-y-true-and-y-pred-when-creating-a-custom-metric-in-keras
# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)

  
#https://stackoverflow.com/questions/50063613/add-loss-function-in-keras
# Custom loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)
        self.target_weights = tf.constant(np.ones((batch_size, max_len)), tf.float32)

    def vae_loss(self, x, x_decoded_mean):
        #xent_loss = K.sum(metrics.categorical_crossentropy(x, x_decoded_mean), axis=-1)
        labels = tf.cast(x, tf.int32)
        #https://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/
        xent_loss = K.sum(tf.contrib.seq2seq.sequence_loss(x_decoded_mean, labels, 
                                                     weights=self.target_weights,
                                                     average_across_timesteps=False,
                                                     average_across_batch=False), axis=-1)#,
                                                     #softmax_loss_function=softmax_loss_f), axis=-1)#,
        #https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        xent_loss = K.mean(xent_loss)
        kl_loss = K.mean(kl_loss)
        return 100*xent_loss + kl_weight * kl_loss
    
    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        print(x.shape, x_decoded_mean.shape)
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)
    
def kl_loss(x, x_decoded_mean):
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    kl_loss = kl_weight * kl_loss
    return kl_loss

#https://github.com/keras-team/keras/issues/5563 # about custom layer
#https://keras.io/layers/writing-your-own-keras-layers/ very important
loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
#https://github.com/keras-team/keras/issues/7395
vae = Model(x, [loss_layer])
opt = Adam(lr=0.01) 
vae.compile(optimizer='adam', loss=[zero_loss], metrics=[kl_loss])
vae.summary()


(?, 15) (?, 15, 20001)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 15, 300)      6000300     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 2048)         10854400    embedding_1[0][0]                
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512)          1049088     bidirectional_1[0][0]            
______________________________________________________________________________________

In [0]:
#block 7 model training and saving
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + model_name + ".h5"  # -{epoch:02d}-{decoded_mean:.2f}
    directory = os.path.dirname(filepath)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)
    return checkpointer

checkpointer = create_model_checkpoint(BASE_DIR+'models', 'vae_lstm_final_2')

np_epochs = 10
for i in range(np_epochs):
    #vae.load_weights(BASE_DIR+'models/vae_lstm_final_2.h5')
    data_train_x,data_train_y, data_val_x, data_val_y = give()
    vae.fit(data_train_x, data_train_y,
       shuffle=True,
       epochs=1,
       batch_size=batch_size,
       validation_data=(data_val_x, data_val_y), callbacks=[checkpointer])
    vae.save(BASE_DIR+'models/vae_lstm_final_2.h5')

    
# build a model to project sentences on the latent space
encoder = Model(x, z_mean)
# build a generator that can sample sentences from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(repeated_context(decoder_input))
_x_decoded_mean = decoder_mean(_h_decoded)
_x_decoded_mean = Activation('softmax')(_x_decoded_mean)
generator = Model(decoder_input, _x_decoded_mean)
encoder.save(BASE_DIR+'models/vae_encoder_final_2.h5')
generator.save(BASE_DIR+'models/vae_generator_final_2.h5')


'''
nb_epoch = 2
n_steps = (800000 / 2) / batch_size  # we use the first 800000

for counter in range(nb_epoch):
    print('-------epoch: ', counter, '--------')
    vae.fit_generator(sent_generator(TRAIN_DATA_FILE, batch_size / 2),
                      steps_per_epoch=n_steps, epochs=1, callbacks=[checkpointer],
                      validation_data=(data_1_val, data_1_val))

vae.fit_generator(sent_generator(TRAIN_DATA_FILE, batch_size / 2),
                      steps_per_epoch=n_steps, epochs=nb_epoch, callbacks=[checkpointer],
                      validation_data=(data_1_val, data_1_val))
'''
#vae.save(BASE_DIR+'models/vae_lstm800k32dim96hid.h5')


Train on 775000 samples, validate on 8000 samples
Epoch 1/1
775000/775000 [==============================] - 1639s 2ms/step - loss: 81.5664 - kl_loss: 1.8406 - val_loss: 191.1367 - val_kl_loss: 1.8303

Epoch 00001: val_loss improved from inf to 191.13672, saving model to drive/ML Project/models/vae_lstm_final.h5
Train on 775000 samples, validate on 8000 samples
Epoch 1/1
775000/775000 [==============================] - 1633s 2ms/step - loss: 68.1229 - kl_loss: 1.7758 - val_loss: 194.0846 - val_kl_loss: 1.7783

Epoch 00001: val_loss did not improve from 191.13672
Train on 775000 samples, validate on 8000 samples
Epoch 1/1
775000/775000 [==============================] - 1635s 2ms/step - loss: 70.6391 - kl_loss: 1.7964 - val_loss: 194.5061 - val_kl_loss: 1.7597

Epoch 00001: val_loss did not improve from 191.13672
Train on 775000 samples, validate on 8000 samples
Epoch 1/1
775000/775000 [==============================] - 1634s 2ms/step - loss: 69.6839 - kl_loss: 1.7776 - val_loss: 194.32

KeyboardInterrupt: ignored

In [0]:
#block 8 loading pretrained model weights
vae.load_weights(BASE_DIR+'models/vae_lstm_final.h5')
encoder = Model(x, z_mean)
# build a generator that can sample sentences from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(repeated_context(decoder_input))
_x_decoded_mean = decoder_mean(_h_decoded)
_x_decoded_mean = Activation('softmax')(_x_decoded_mean)
generator = Model(decoder_input, _x_decoded_mean)
encoder.save(BASE_DIR+'models/vae_encoder_final.h5')
generator.save(BASE_DIR+'models/vae_generator_final.h5')

KeyboardInterrupt: ignored

In [0]:
#block 9 testing functions
encoder.load_weights(BASE_DIR+'models/vae_encoder.h5')
generator.load_weights(BASE_DIR+'models/vae_generator.h5')

def print_sentence(sent_vect):
    word_list = list(np.vectorize(index2word.get)(sent_vect))
    w_list = [w for w in word_list] # if w not in ['pad']]
    print(' '.join(w_list))
    #print(word_list)
index2word = {v: k for k, v in word_index.items()}
index2word[0] = 'pad'
'''
#test on a validation sentence
sent_idx = 100
print_sentence(data_val[sent_idx])
#print(data_val[sent_idx])
sent_encoded = encoder.predict(data_val[sent_idx:sent_idx+2,:])
x_test_reconstructed = generator.predict(sent_encoded, batch_size = 1)
reconstructed_indexes = np.apply_along_axis(np.argmax, 1, x_test_reconstructed[0])
np.apply_along_axis(np.max, 1, x_test_reconstructed[0])
np.max(np.apply_along_axis(np.max, 1, x_test_reconstructed[0]))
word_list = list(np.vectorize(index2word.get)(reconstructed_indexes))
print(' '.join(word_list))
original_sent = list(np.vectorize(index2word.get)(data_val[sent_idx]))
print(' '.join(original_sent))
'''

# function to parse a sentence
def sent_parse(sentence, mat_shape):
    sequence = tokenizer.texts_to_sequences(sentence)
    padded_sent = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post',truncating='post')
    return padded_sent#[padded_sent, sent_one_hot]


# input: encoded sentence vector
# output: encoded sentence vector in dataset with highest cosine similarity
'''
def find_similar_encoding(sent_vect):
    all_cosine = []
    for sent in sent_encoded:
        result = 1 - spatial.distance.cosine(sent_vect, sent)
        all_cosine.append(result)
    data_array = np.array(all_cosine)
    maximum = data_array.argsort()[-3:][::-1][1]
    new_vec = sent_encoded[maximum]
    return new_vec
'''
def find_similar_encoding(sent_vect):
    dis = 100
    global ret
    for sent in encoder.predict(data_train):
        result = 1 - spatial.distance.cosine(sent_vect, sent)
        if(result<dis):
          dis = result
          ret = sent
    return ret

# input: two points, integer n
# output: n equidistant points on the line between the input points (inclusive)
def shortest_homology(point_one, point_two, num):
    dist_vec = point_two - point_one
    sample = np.linspace(0, 1, num, endpoint = True)
    hom_sample = []
    for s in sample:
        hom_sample.append(point_one + s * dist_vec)
    return hom_sample



# input: original dimension sentence vector
# output: sentence text
def print_latent_sentence(sent_vect):
    sent_vect = np.reshape(sent_vect,[1,latent_dim])
    sent_reconstructed = generator.predict(sent_vect)
    sent_reconstructed = np.reshape(sent_reconstructed,[max_len,NB_WORDS])
    reconstructed_indexes = np.apply_along_axis(np.argmax, 1, sent_reconstructed)
    word_list = list(np.vectorize(index2word.get)(reconstructed_indexes))
    w_list = [w for w in word_list if w not in ['pad']]
    print(' '.join(w_list))
    #print(word_list)
    
       
        
def new_sents_interp(sent1, sent2, n):
    tok_sent1 = sent_parse(sent1, [30])
    tok_sent2 = sent_parse(sent2, [30])
    enc_sent1 = encoder.predict(tok_sent1, batch_size = 16)
    enc_sent2 = encoder.predict(tok_sent2, batch_size = 16)
    test_hom = shortest_homology(enc_sent1, enc_sent2, n)
    for point in test_hom:
        print_latent_sentence(point)


print("intepolation starts")
#====================== Example ====================================#
sentence1=['where can i find a book on machine learning']
mysent = sent_parse(sentence1, [27])
mysent_encoded = encoder.predict(mysent, batch_size = 16)
print_latent_sentence(mysent_encoded)
#print_latent_sentence(find_similar_encoding(mysent_encoded))

sentence2=['how can i become a successful entrepreneur']
mysent2 = sent_parse(sentence2, [27])
mysent_encoded2 = encoder.predict(mysent2, batch_size = 16)
#print_latent_sentence(mysent_encoded2)
#print_latent_sentence(find_similar_encoding(mysent_encoded2))
#print('-----------------')

new_sents_interp(sentence1, sentence2, 6)


intepolation starts
where can i find a book on machine learning
where can i find a book on machine learning
where can i find a book on machine learning
where can i find a successful on machine
how can i become a successful developer hacker
how can i become a successful entrepreneur
how can i become a successful entrepreneur


In [0]:
#block 10 sample tests
sentence1 =['what is java pad how to pad java programming language']
sentence2 =['how can you pad a blu ray dvd on a regular dvd player']
sentence3 =['How does pad printing work?']
def fill_pad(sentence):
  mysent = sent_parse(sentence, [27])
  encoded = encoder.predict(mysent, batch_size = 16)
  print_latent_sentence(encoded)
fill_pad(sentence3)

how does the printing work


In [0]:
print_sentence(data_train_y[103])
print_sentence(data_train_x[102])

how can you play a blu ray dvd on a regular dvd player pad pad
will a blu ray play on a regular dvd player if so how pad pad


In [0]:
|'how can i pad the speed of my internet connection while using a vpn'

In [0]:
#'where can i pad some water'